### Copyright-protected material, all rights reserved. (c) University of Vienna.
_Copyright Notice of the corresponding course at Moodle applies. <br> Only to be used in the MRE course._

# MRE Assignment 1 - Digital Image Processing

In this assignment you will use Python (using Pillow or OpenCV) to load, transcode and store images. You will also use the libraries to extract some basic metadata from the images and store them in a data structure for easy access.

In this notebook, you will implement your solution. This notebook will be imported into the "*_def.ipynb" notebook.

Of course you can include code for testing your implementation in this implementation notebook, but code for testing and output generated for testing is not going to be assessed.

Of course, your code for the solutions in this notebook will be inspected and is subject to grading.


## Setup

For general installation instructions, please refer to the ressources given for all the assignments [in Moodle](https://moodle.univie.ac.at/course/view.php?id=260637#section-13).

If the cell below executes without error, you can start the assignment!

In [35]:
# -------- Imports --------
# Please do not change the contents of this cell!

# In case you work in a local environment on your own machine,
# how to install the required packages:
#   PIP:     pip install <name> / pip install --upgrade <name>
#   CONDA:   conda install -c conda-forge <name>  /  conda install -c anaconda <name>


# Imports required by us.
import cv2                             # opencv-python
from PIL import Image                  # pillow
from PIL.ExifTags import TAGS
from matplotlib import pyplot as plt   # matplotlib
from sklearn.metrics.pairwise import cosine_similarity  # scikit-learn
import numpy as np                     # numpy
import pandas as pd                    # pandas
from IPython.display import display    # packaged with python

# This directive will allow matplotlib to render interactive plots in the notebook.
%matplotlib widget

In the cells below, place your own imports, global variables, (helper) functions and classes. Feel free to add cells here as you see fit.

In [36]:
# Please place your own imports here.
import os
from PIL import ImageFilter

In [37]:
# Place any helper functions, global variables and classes here.

## Task 1.1: Image formats transformation and adding filters


In [38]:
# Write your function here.

# Transcodes images to JPEG format of adjustable quality (for JPEG).
def JPEGImageConverter(inputImg: str, outputDir: str, quality: float = 1.0) -> None:
    os.makedirs(outputDir, exist_ok=True)   # make output directory if not existant
    
    if os.path.isdir(inputImg):
        for filename in os.listdir(inputImg):
            filepath = os.path.join(inputImg, filename)
            
            fileName, fileExtension = os.path.splitext(filename)
            if os.path.isfile(filepath) and fileExtension.upper() in ['.JPG', '.JPEG', '.BMP', '.PNG', '.GIF']: #just take images
                convertFile(filepath, outputDir, quality)
    else:
        convertFile(inputImg, outputDir, quality)

def convertFile(inputImg: str, outputDir: str, quality: float = 1.0):
    with Image.open(inputImg) as image:
        originalName = os.path.basename(inputImg)
        fileName, fileExtension = os.path.splitext(originalName)
        
        if quality == 1.0:
            outputName = f"{originalName}.jpg"
        else:
            outputName = f"{originalName}-{quality}.jpg" 
            
        outputPath = os.path.join(outputDir, outputName)
                
        image.convert("RGB").save(outputPath, format="JPEG", quality=int(quality * 100))

    

In [39]:
# Test your function here.
#JPEGImageConverter("images/jawa-1.jpg", "./output_dir", 0.77)

In [40]:
# Add Blur filter
def BlurImage(inputImg: str) -> None:
    with Image.open(inputImg) as image:
        originalName = os.path.basename(inputImg)
        fileName, fileExtension = os.path.splitext(originalName)
        outputName = f"{originalName}-blurred.jpg"
        outputPath = os.path.join("./output_dir", outputName)
        
        blurred = image.filter(ImageFilter.BLUR)
        blurred.convert("RGB").save(outputPath, format="JPEG", quality=int(100))

In [41]:
# Test your function here.
#BlurImage("images/zebra3m.jpg")

## Task 1.2:  Extract / Get Image Metadata

In [42]:
# Write your function here. 

# Extracts metadata from an image and places them into a Pandas DataFrame.
def ImageMetadataGenerator(inputImage: str) -> pd.DataFrame:
    pass

In [43]:
# Test your function here.

## Task 1.3: Drawing circles around objects in an image

In [44]:
# Write your function here. 

# Identifies shapes in an images and draws circles around them.
def IdentifyObjects(inputImage: str) -> None:
    pass

In [45]:
# Test your function here.

## Task 1.4: Duplicate image finder using cosine similarity

In [46]:
# Calculates a histogram for one image.
def ImageHistogramGenerator(inputImage: str) -> np.array:
    pass

In [47]:
# Test your function here.

In [48]:
# Find duplicate images and return their similarity scores in a dataframe.
def FindDuplicateImages(inputDir1 : str, inputDir2: str, similarityThreshold: float = 1.0) -> pd.DataFrame:
    pass

In [49]:
# Test your function here.